**Задача 1** Обучите FasterRCNN на датасете PascalVOC.

Сколько классов в PascalVOC?

Как записан target?

Вызывает ли датасет VOCDetection ошибки при чтении? Если да, то как их избежать?

In [ ]:
from torchvision.datasets import VOCDetection, VisionDataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models import mobilenet_v2, resnet34
import torch.nn.functional as F
import torch.optim as optim
import os

In [ ]:
transforms_ = transforms.Compose([transforms.ToTensor()])

In [ ]:
voc = VOCDetection(root='Voc',download=True, transform=transforms_)

Using downloaded and verified file: Voc/VOCtrainval_11-May-2012.tar


In [ ]:
voc_dataloader = DataLoader(dataset=voc, batch_size=1)

In [ ]:
sample = next(iter(voc_dataloader))
print(sample)

## Оборачиваем VOCDetection

In [ ]:
print(os.listdir(os.path.join(os.getcwd(), 'Voc/', 'VOCdevkit/', 'VOC2012')))

['SegmentationClass', 'SegmentationObject', 'ImageSets', 'JPEGImages', 'Annotations']


In [ ]:
path2data = r'./Voc'

In [ ]:
classes2labels = {
        "__background__ ":0,
        "aeroplane":1,
        "bicycle":2,
        "bird":3,
        "boat":4,
        "bottle":5,
        "bus":6,
        "car":7,
        "cat":8,
        "chair":9,
        "cow":10,
        "diningtable":11,
        "dog":12,
        "horse":13,
        "motorbike":14,
        "person":15,
        "pottedplant":16,
        "sheep":17,
        "sofa":18,
        "train":19,
        "tvmonitor":20,
}

In [ ]:
class MyVOCDetection(VOCDetection):
  def __init__(self, root, transforms_=None):
    super().__init__(root, download=False)
    self.transforms_ = transforms_
  
  def __getitem__(self, index):
    image, target = super().__getitem__(index)

    labels = []
    boxes = []
    for object_ in target['annotation']['object']:
        labels.append(classes2labels[object_['name']])
        boxes.append([int(object_['bndbox']['xmin']), int(object_['bndbox']['ymin']), int(object_['bndbox']['xmax']), int(object_['bndbox']['ymax'])])
    boxes = torch.as_tensor(boxes)
    target = {'boxes':boxes,'labels':labels}

    if self.transforms_ is not None:
      image = self.transforms_(image)

    return image, target

In [ ]:
transforms_ = transforms.Compose([ transforms.ToTensor(),
                          transforms.RandomHorizontalFlip()])

In [ ]:
VOC_dataset = MyVOCDetection(path2data, transforms_=None)

Using downloaded and verified file: ./Voc/VOCtrainval_11-May-2012.tar


In [ ]:
VOC_dataset[0]

(<PIL.Image.Image image mode=RGB size=500x442 at 0x7FB4080EBEF0>,
 {'boxes': tensor([[ 53,  87, 471, 420],
          [158,  44, 289, 167]]), 'labels': tensor([13, 15])})

## Совместное преобразование

In [ ]:
class MyVOCDetection(VOCDetection):
  def __init__(self, root, transforms_=None):
    super().__init__(root, download=True)
    self.transforms_ = transforms_
  
  def __getitem__(self, index):
    image, target = super().__getitem__(index)

    labels = []
    boxes = []
    for object_ in target['annotation']['object']:
        labels.append(classes2labels[object_['name']])
        boxes.append([int(object_['bndbox']['xmin']), int(object_['bndbox']['ymin']), int(object_['bndbox']['xmax']), int(object_['bndbox']['ymax'])])
    boxes = torch.as_tensor(boxes)
    labels = torch.as_tensor(labels)
    target = {'boxes':boxes,'labels':labels}

    print('image befor transform', type(image))
    print('target before transform', target)
    
    if self.transforms_ is not None:
      image, target = self.transforms_(image, target)
    print('target after transform', target)
    print('image after transform', type(image))
    return image, target

In [ ]:
transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.5)
    ])

In [ ]:
VOC_dataset = MyVOCDetection(path2data, transforms_=transforms_)

Using downloaded and verified file: ./Voc/VOCtrainval_11-May-2012.tar


### Трюк с версиями torchvision




In [ ]:
VOC_dataset[0]

image befor transform <class 'PIL.Image.Image'>
target before transform {'boxes': tensor([[ 53,  87, 471, 420],
        [158,  44, 289, 167]]), 'labels': [13, 15]}


TypeError: ignored

In [ ]:
torchvision.__version__

'0.6.0+cu101'

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 9284 (delta 9), reused 4 (delta 1), pack-reused 9258
Receiving objects: 100% (9284/9284), 11.25 MiB | 6.50 MiB/s, done.
Resolving deltas: 100% (6431/6431), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be37608 version check against PyTorch's CUDA version


In [ ]:
import transforms as T

In [ ]:
transforms_ = T.Compose([
           T.ToTensor(),
           T.RandomHorizontalFlip(0.5)
])

In [ ]:
VOC_dataset = MyVOCDetection(path2data, transforms_=transforms_)

Using downloaded and verified file: ./Voc/VOCtrainval_11-May-2012.tar


In [ ]:
image, target = VOC_dataset[0]

image befor transform <class 'PIL.Image.Image'>
target before transform {'boxes': tensor([[ 53,  87, 471, 420],
        [158,  44, 289, 167]]), 'labels': [13, 15]}
target after transform {'boxes': tensor([[ 29,  87, 447, 420],
        [211,  44, 342, 167]]), 'labels': [13, 15]}
image after transform <class 'torch.Tensor'>


In [ ]:
VOC_dataloader = DataLoader(dataset=VOC_dataset, batch_size=1)
next(iter(VOC_dataloader))

image befor transform <class 'PIL.Image.Image'>
target before transform {'boxes': tensor([[ 53,  87, 471, 420],
        [158,  44, 289, 167]]), 'labels': [13, 15]}
target after transform {'boxes': tensor([[ 29,  87, 447, 420],
        [211,  44, 342, 167]]), 'labels': [13, 15]}
image after transform <class 'torch.Tensor'>


[tensor([[[[0.8000, 0.7961, 0.7922,  ..., 1.0000, 1.0000, 1.0000],
           [0.7922, 0.7961, 0.7961,  ..., 1.0000, 1.0000, 1.0000],
           [0.8039, 0.8039, 0.8078,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [0.4667, 0.4706, 0.5373,  ..., 0.8078, 0.7608, 0.8039],
           [0.4784, 0.4235, 0.4353,  ..., 0.7569, 0.7569, 0.7529],
           [0.3137, 0.2863, 0.3216,  ..., 0.7255, 0.7059, 0.6980]],
 
          [[0.8118, 0.8078, 0.8039,  ..., 1.0000, 1.0000, 1.0000],
           [0.8039, 0.8078, 0.8078,  ..., 1.0000, 1.0000, 1.0000],
           [0.8157, 0.8157, 0.8196,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [0.4431, 0.4392, 0.5020,  ..., 0.7529, 0.7059, 0.7490],
           [0.4667, 0.4039, 0.4157,  ..., 0.7020, 0.7020, 0.6980],
           [0.3020, 0.2667, 0.3020,  ..., 0.6706, 0.6510, 0.6431]],
 
          [[0.9804, 0.9765, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
           [0.9725, 0.9765, 0.9765,  ..., 1.0000, 1.0000, 1.0000],
           [0.9843, 0.98

In [ ]:
class MyVOCDetection(VOCDetection):
  def __init__(self, root, transforms_=None):
    super().__init__(root, download=True)
    self.transforms_ = transforms_
  
  def __getitem__(self, index):
    image, target = super().__getitem__(index)

    labels = []
    boxes = []
    for object_ in target['annotation']['object']:
        labels.append(classes2labels[object_['name']])
        boxes.append([int(object_['bndbox']['xmin']), int(object_['bndbox']['ymin']), int(object_['bndbox']['xmax']), int(object_['bndbox']['ymax'])])
    boxes = torch.as_tensor(boxes)
    labels = torch.as_tensor(labels)
    target = {'boxes':boxes,'labels':labels}

    if self.transforms_ is not None:
      image, target = self.transforms_(image, target)
    return image, target

In [ ]:
VOC_dataset = MyVOCDetection(path2data, transforms_=transforms_)

Using downloaded and verified file: ./Voc/VOCtrainval_11-May-2012.tar


In [ ]:
image, target = VOC_dataset[3]
image.size()

torch.Size([3, 500, 333])

## Сопоставление изображений разных масштабов

In [ ]:
VOC_dataloader = DataLoader(dataset=VOC_dataset, batch_size=2)
example = next(iter(VOC_dataloader))

RuntimeError: ignored

In [ ]:
def collate_fn(batch):
    return(tuple(zip(*batch)))

In [ ]:
VOC_dataloader = DataLoader(dataset=VOC_dataset, batch_size=2, num_workers=4, collate_fn=collate_fn)
example = next(iter(VOC_dataloader))

In [ ]:
ex_images, ex_targets = next(iter(VOC_dataloader))
ex_images = ex_images
print('image')
print(ex_images)
print('Так записан target:')
print(ex_targets)

image
(tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.7922, 0.7961, 0.8000],
         [1.0000, 1.0000, 1.0000,  ..., 0.7961, 0.7961, 0.7922],
         [1.0000, 1.0000, 1.0000,  ..., 0.8078, 0.8039, 0.8039],
         ...,
         [0.8039, 0.7608, 0.8078,  ..., 0.5373, 0.4706, 0.4667],
         [0.7529, 0.7569, 0.7569,  ..., 0.4353, 0.4235, 0.4784],
         [0.6980, 0.7059, 0.7255,  ..., 0.3216, 0.2863, 0.3137]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.8039, 0.8078, 0.8118],
         [1.0000, 1.0000, 1.0000,  ..., 0.8078, 0.8078, 0.8039],
         [1.0000, 1.0000, 1.0000,  ..., 0.8196, 0.8157, 0.8157],
         ...,
         [0.7490, 0.7059, 0.7529,  ..., 0.5020, 0.4392, 0.4431],
         [0.6980, 0.7020, 0.7020,  ..., 0.4157, 0.4039, 0.4667],
         [0.6431, 0.6510, 0.6706,  ..., 0.3020, 0.2667, 0.3020]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.9725, 0.9765, 0.9804],
         [1.0000, 1.0000, 1.0000,  ..., 0.9765, 0.9765, 0.9725],
         [1.0000, 1.0000, 1.0000,  ..., 0.9882, 0.9

In [ ]:
print(ex_images[0].size())
print(ex_images[1].size())


torch.Size([3, 442, 500])
torch.Size([3, 327, 500])


In [ ]:
target = ex_targets[0]

In [ ]:
{k:v.cuda() for k,v in target.items()}

{'boxes': tensor([[ 53,  87, 471, 420],
         [158,  44, 289, 167]], device='cuda:0'),
 'labels': tensor([13, 15], device='cuda:0')}

In [ ]:
images = [image.cuda() for image in ex_images]

In [ ]:
images = [image.cuda() for image in images]
targets = [{k:v.cuda() for k,v in target.items()} for target in targets]

In [ ]:
print(images)
print(targets)

[tensor([[[0.8000, 0.7961, 0.7922,  ..., 1.0000, 1.0000, 1.0000],
         [0.7922, 0.7961, 0.7961,  ..., 1.0000, 1.0000, 1.0000],
         [0.8039, 0.8039, 0.8078,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.4667, 0.4706, 0.5373,  ..., 0.8078, 0.7608, 0.8039],
         [0.4784, 0.4235, 0.4353,  ..., 0.7569, 0.7569, 0.7529],
         [0.3137, 0.2863, 0.3216,  ..., 0.7255, 0.7059, 0.6980]],

        [[0.8118, 0.8078, 0.8039,  ..., 1.0000, 1.0000, 1.0000],
         [0.8039, 0.8078, 0.8078,  ..., 1.0000, 1.0000, 1.0000],
         [0.8157, 0.8157, 0.8196,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.4431, 0.4392, 0.5020,  ..., 0.7529, 0.7059, 0.7490],
         [0.4667, 0.4039, 0.4157,  ..., 0.7020, 0.7020, 0.6980],
         [0.3020, 0.2667, 0.3020,  ..., 0.6706, 0.6510, 0.6431]],

        [[0.9804, 0.9765, 0.9725,  ..., 1.0000, 1.0000, 1.0000],
         [0.9725, 0.9765, 0.9765,  ..., 1.0000, 1.0000, 1.0000],
         [0.9843, 0.9843, 0.9882,  ..., 1.0000, 1.0000, 1

## Создаем сеть FasterRCNN

In [ ]:
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

In [ ]:
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=["0"], output_size=7, sampling_ratio=2)

In [ ]:
backbone = resnet34(pretrained=True)


NameError: ignored

In [ ]:
print(backbone)

In [ ]:
backbone = nn.Sequential(*list(backbone.children())[:-1])

In [ ]:
print(backbone)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
backbone.out_channels = 512

In [ ]:
model = FasterRCNN(backbone,
                   num_classes=21,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler).cuda()
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
       

In [ ]:
ex_images, ex_targets = next(iter(VOC_dataloader))

In [ ]:
img = ex_images[0]
img = img.cuda()
print(img.size())

torch.Size([3, 442, 500])


In [ ]:
img = img.unsqueeze(0)
print(img.size())

torch.Size([1, 3, 442, 500])


In [ ]:
model(img)

## Обучаем сеть

In [ ]:
model.train()
for images, targets in VOC_dataloader:
    images = [image.cuda() for image in images]
    targets = [{k:v.cuda() for k,v in target.items()} for target in targets]
    losses_dict = model(images, targets)
    print(losses)
    break

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


{'loss_classifier': tensor(3.1233, device='cuda:0', grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.0029, device='cuda:0', grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.6893, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(14.6351, device='cuda:0', grad_fn=<DivBackward0>)}


Суммируем лоссы

In [ ]:
model.train()
for images, targets in VOC_dataloader:
    images = [image.cuda() for image in images]
    targets = [{k:v.cuda() for k,v in target.items()} for target in targets]
    losses_dict = model(images, targets)
    losses_sum = sum(loss for loss in losses_dict.values())

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

In [ ]:
model.train()
for images, targets in VOC_dataloader:
    model.zero_grad()
    images = [image.cuda() for image in images]
    targets = [{k:v.cuda() for k,v in target.items()} for target in targets]
    losses_dict = model(images, targets)
    losses_sum = sum(loss for loss in losses_dict.values())
    losses_sum.backward()
    optimizer.step()

### Optimizer.zero_grad()

In [ ]:
model.train()
for images, targets in VOC_dataloader:
    optimizer.zero_grad()
    images = [image.cuda() for image in images]
    targets = [{k:v.cuda() for k,v in target.items()} for target in targets]
    losses_dict = model(images, targets)
    losses_sum = sum(loss for loss in losses_dict.values())
    losses_sum.backward()
    optimizer.step()

## Более простое решение

In [ ]:
model = fasterrcnn_resnet50_fpn(pretrained=True).cuda()
model.eval()

In [ ]:
model.train()


In [ ]:
print(ex_target)
losses = model(ex_image, ex_target)


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches


fig, ax = plt.subplots(1)
ax.imshow(ex_image.cpu().data[0].permute(1,2,0).numpy())

boxes = pred['boxes'].data.cpu().numpy().tolist()
labels = pred['labels'].data.cpu().numpy().tolist()
scores = pred['scores'].data.cpu().numpy().tolist()

for box, label, score in zip(boxes, labels, scores):
    if score > 0.9:
        # label = coco_cats[label]['name']
        left, bot, right, top = box
        x, y, w, h = [val for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        #ax.text(x, y, "{}, {:.2f}%".format(label, score*100), bbox=dict(facecolor='white', alpha=0.5))
        ax.text(x, y, str(label), bbox=dict(facecolor='white', alpha=0.5))

# Домашнее задание

**Задача 0** Ещё раз разобраться с кодом для обучения FastRCNN

**Задача 1** Напишите реализацию алгоритма расчета Intersection over Union. 
1. Функция принимает на вход координаты (x,y левого нижнего угола, x, y правого верхнего угла) для двух обрамляющийх прямоугольников (bounding boxes) и возвращает меру IoU — отношение их пересечения к общей площади.
2. Функция принимает на вход координаты (x,y центра и h,w высота и ширина) для двух обрамляющийх прямоугольников (bounding boxes) и возвращает меру IoU — отношение их пересечения к общей площади.

**Задача 2** Напишите реализацию алгоритма расчета Non-Maximum Suppression. Алгоритм принимает на вход набор обрамляющих прямоугольников (bounding box'ов) и соответствующих им уверенностей в оценке класса (confidence). Нужно вернуть набор прямоугольников, IoU которых не больше заданного порога (threshold), оставляя только те, в которых сеть уверена больше.

**Задача 3** Дан датасет с 10 классами. Выход YOLO имеет вид:
* A [30,30,40,50] 0.7 [0.01, 0.01, 0.02, 0.05, 0.85, 0.01, 0.01, 0.01, 0.01, 0.01]
* B [30,30,50,40] 0.8 [0.01, 0.01, 0.02, 0.05, 0.85, 0.01, 0.01, 0.01, 0.01, 0.01]
* C [20,20, 30,30] 0.8 [0.01, 0.01, 0.84, 0.05, 0.03, 0.01, 0.01, 0.01, 0.01, 0.01]
* D [70,30,50,40] 0.7 [0.01, 0.01, 0.01, 0.01, 0.9, 0.01, 0.01, 0.01, 0.01, 0.01]
* E [70,30,40,50] 0.6 [0.01, 0.01, 0.01, 0.01, 0.9, 0.01, 0.01, 0.01, 0.01, 0.01]

Напишите, какие bounding boxes останутся после Non-Maximum Supression. Порог 0.5

**Задача 4** Дан датасет с 10 классами: собака, кошка, рюкзак, велосипед, человек, самолет, телевизор, автомобиль, ваза, картина.
Выход YOLO имеет вид:

* A [30,30,40,50] 0.7 [0.01, 0.01, 0.02, 0.05, 0.85, 0.01, 0.01, 0.01, 0.01, 0.01]
* B [30,30,50,40] 0.8 [0.01, 0.01, 0.02, 0.05, 0.85, 0.01, 0.01, 0.01, 0.01, 0.01]
* C [20,20, 30,30] 0.8 [0.01, 0.01, 0.84, 0.05, 0.03, 0.01, 0.01, 0.01, 0.01, 0.01]
* D [70,30,50,40] 0.7 [0.01, 0.01, 0.01, 0.01, 0.9, 0.01, 0.01, 0.01, 0.01, 0.01]
* E [70,30,40,50] 0.6 [0.01, 0.01, 0.01, 0.01, 0.9, 0.01, 0.01, 0.01, 0.01, 0.01]


1. Какой из bounding box'ов лучше соответствует классу "человек"?
2. Какой из bounding box'ов лучше соответствует классу "рюкзак"?

**Задача 5** Нарисуйте архитектуру сети, которую можно записать формулой:
1. f(f(x + f(f(x + f(f(x))))
2. g(g(g(f(f(x + f(f(x + f(f(x)))))))
3. g(g(g(f(f(f(x))))+f(f(x)) + f(x))

Запишите для этих сетей производную.

**Задача 6** Какой будет размер вектора y, который получается при умножении вектора v на матрицу A слева (y = Av):
1. если A — матрица 3x7, а v — вектора семи чисел
2. если A — матрица 7x3, а v — вектора трех чисел

**Задача 7** Дана сеть:


In [ ]:
class Head(nn.Module):
    def __init__(self,in_channels, num_classes, num_boxes, s1, s2, s3):
        self.num_classes = num_classes
        self.num_boxes = num_boxes
        self.in_channels = in_channels
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3

        self.A = torch.Conv2d(in_channels,in_channels*2,kernel_size=3, stride=2, padding=1)
        self.B = torch.Conv2d(in_channels*2,in_channels*2,kernel_size=3, stride=2, padding=1)
        self.C = torch.Conv2d(in_channels*2,in_channels*4,kernel_size=3, stride=2, padding=1)
        self.D = torch.Conv2d(in_channels*4,in_channels*4,kernel_size=3, stride=2, padding=1)
        self.E = torch.Conv2d(in_channels*4,in_channels*8,kernel_size=3, stride=2, padding=1)
        self.F = torch.Conv2d(in_channels*8,in_channels*8,kernel_size=3, stride=2, padding=1)

        self.linear_B = nn.Linear(?, num_classes)
        self.linear_D = nn.Linear(?, num_classes)
        self.linear_F = nn.Linear(?, num_classes)
    
    def forward(self,x):
        x = F.relu(self.A(x))
        out_B = F.relu(self.B(x))
        
        x = F.relu(self.C(out_B))
        out_D = F.relu(self.D(x))

        x = F.relu(self.E(out_D))
        out_F = F.relu(self.F(x))

        out_B = out_B.view(out_B.size(0),-1)
        out_B = self.linear_B(out_B)

        out_D = out_B.view(out_D.size(0),-1)
        out_D = self.linear_D(out_D)

        out_F = out_D.view(out_F.size(0),-1)
        out_F = self.linear_F(out_F)

        return F.softmax(B), F.softmax(D), F.softmax(F)

1. Изобразите её структуру.
2. На вход подается изображение 100x64x64. Какого размера будет изображение после слоев B, D, F?
3. Какие параметры нужно указать в слоях linear_B, linear_D, linear_F? 

**Задача 8**\* Напишите датасет, который берет либо изображения из одной и той же папки либо изображения из разных папок и вобзвращает пару изображений и значение t=0, если они взяты из одной папки, t=1, если взяты из разных папок.

**Задача 9**\* Напишите датасет, который для каждого изображения из одной папки берет изображение из той же папки и изображение из любой другой папки и возвращает все три изображения.